In [2]:
import awkward as ak
import numpy as np
import pickle as pkl
import hist 
import matplotlib.pyplot as plt
import uproot

import os
from numba import njit

In [9]:
RAW_DATA_DIR = '/Users/ravikoka/repos/z_plus_hf/feasibility/data/multi/run1/'

Z_counter = 0
W_counter = 0

events_w_Z = ak.Array([])
for file in os.listdir(RAW_DATA_DIR):
    
    with open(RAW_DATA_DIR+ file, 'rb') as in_file:
        events = pkl.load(in_file)
        
        z_counts = np.sum(events['pdg'] == 23)
        W_counts = np.sum(events['pdg'] == abs(24))
        
        if z_counts > 0:
            #temp = events
            
            contains_Z = ak.any(events['pdg']==23, axis=1)
            events_w_Z = ak.concatenate([events_w_Z, events[contains_Z]])
        
        Z_counter += z_counts
        
        W_counter += W_counts

In [10]:
events

<Array [{px: [0, ...], py: [...], ...}, ...] type='500 * {px: var * float64...'>

In [6]:
W_counter

np.int64(0)

In [7]:
W_counter

np.int64(0)

In [8]:
Z_counter

np.int64(0)

In [84]:
ak.any(events_w_Z['pdg']==23, axis=1)

<Array [True, True, True, True, ..., True, True, True, True] type='11 * bool'>

In [91]:
events_zipped = ak.zip(
    {
        'px': events_w_Z['px'],
        'py': events_w_Z['py'],
        'pz': events_w_Z['pz'],
        'pdg': events_w_Z['pdg'],
        'mother_list': events_w_Z['mother_list'],
    }, 
    depth_limit=2
)

In [92]:
events_zipped

<Array [[{px: 0, py: 0, ...}, ..., {...}], ...] type='11 * var * {px: float...'>

In [105]:
contains_b = ak.any(events_zipped['pdg']==5, axis=1)
contains_b

<Array [False, False, False, False, ..., False, False, True] type='11 * bool'>

In [107]:
events_w_Z_and_b = events_zipped[contains_b]
events_w_Z_and_b

<Array [[{px: 0, py: 0, pz: ..., ...}, ...]] type='1 * var * {px: float64, ...'>

In [115]:
#events_w_Z_and_b['pdg'] == 5

for i, pdg in enumerate(ak.flatten(events_w_Z_and_b['pdg'])):
    if pdg == 5:
        print(f'particle num {i}, b quark')

particle num 566, b quark
particle num 568, b quark
particle num 573, b quark
particle num 577, b quark
particle num 792, b quark


In [102]:
np.sum(events_zipped['pdg'] == 5)

np.int64(5)

In [95]:
Z_bosons = events_zipped[events_zipped['pdg'] == 23]
Z_bosons

<Array [[{px: -58.7, py: ..., ...}, ...], ...] type='11 * var * {px: float6...'>

In [97]:
prompt_Z_bosons = Z_bosons[:, 0]

In [100]:
pt_hist = hist.Hist(hist.axis.Regular(60, 0, 100, name='Z0 pt'))
pt_hist.fill(ak.flatten(Z_bosons['px']**2) + ak.flatten(Z_bosons['py']**2))

Hist(Regular(60, 0, 100, name='Z0 pt'), storage=Double()) # Sum: 27.0 (67.0 with flow)

# old

In [43]:
temp

<Array [{px: [0, ...], py: [...], ...}, ...] type='500 * {px: var * float64...'>

In [44]:
any = ak.any(temp['pdg']==23, axis=1)
any

<Array [False, False, False, False, ..., False, False, False] type='500 * bool'>

In [45]:
events_w_Z = temp[any]

In [79]:
eee = events_w_Z['mother_list'] == events_zipped['mother_list']

In [80]:
np.sum(~eee)

np.int64(0)

In [73]:
events_zipped = ak.zip(
    {
        'px': events_w_Z['px'],
        'py': events_w_Z['py'],
        'pz': events_w_Z['pz'],
        'pdg': events_w_Z['pdg'],
        'mother_list': events_w_Z['mother_list'],
    }, 
    depth_limit=1
)

In [76]:
events_zipped['mother_list']

<Array [[[], [], [6], ..., [1324], [1324]]] type='1 * var * var * int64'>

In [53]:
print(ak.flatten(events_w_Z['pdg']))

[2212, 2212, 1, -1, 23, 1, -1, 23, 21, ..., 111, 22, 22, 310, 22, 22, 211, -211]


In [48]:
events_w_Z['pdg'][events_w_Z['pdg'] == 23]

<Array [[23, 23, 23, 23, 23]] type='1 * var * int64'>

In [49]:
events_w_Z['pz'][events_w_Z['pdg'] == 23]

<Array [[1.29e+03, 1.17e+03, ..., 1.17e+03, 1.17e+03]] type='1 * var * float64'>

In [50]:
events_w_Z['mother_list'][events_w_Z['pdg'] == 23]

<Array [[[3, 4], [5], [8], [61], [116]]] type='1 * var * var * int64'>

In [37]:
np.sum(temp[any]['pdg'] == 23)

np.int64(5)

In [27]:
condition = temp['pdg'] == 23
temp[condition]
#ak.where(condition, temp, None)
#temp[condition]

<Array [{px: [], py: [], pz: [], ...}, ...] type='500 * {px: var * float64,...'>

In [24]:
ak.flatten(temp['pdg'][temp['pdg'] == 23])

<Array [23, 23, 23, 23, 23] type='5 * int64'>

In [25]:
ak.max(temp['pdg'], axis=1)

<Array [2214, 3312, 3322, 3212, ..., 3224, 3322, 3312] type='500 * ?int64'>

In [6]:
Z_counter

np.int64(67)